In [2]:
# https://www.youtube.com/watch?v=Y7XBsFzByTQ

import sys
import cv2
import os
import numpy as np
import pytesseract
import matplotlib.pyplot as plt

# button dimensions (y1,y2,x1,x2)
images=['frame','inverted','ycrcb','hsvim','skin_test','equally_saturate','equally_hue','skinRegionHSV','masked_from_hsvim','masked_from_frame','blurred','thresh_before_contour','thresh_before_convexHull','before_defects','thresh']
control = 0
rgb=0
normalized_on=False

button_width = 100
button = [20,40,0,button_width]
button2 = [20,40,button_width,button_width*2]
button3 = [20,40,button_width*2,button_width*3]
button4 = [20,40,button_width*3,button_width*4]

def drawButton(img,name,mat):
    img[mat[0]:mat[1],mat[2]:mat[3]] = 180
    cv2.putText(img, name,(mat[2],35),cv2.FONT_HERSHEY_PLAIN, 2,(0),3)
    
#https://stackoverflow.com/questions/55919337/creating-capture-button-on-window
# function that handles the mousclicks
def process_click(event, x, y,flags, params):
    # check if the click is within the dimensions of the button
    global control
    global images
    global rgb
    global normalized_on
    if event == cv2.EVENT_LBUTTONDOWN:
        if y > button[0] and y < button[1] and x > button[2] and x < button[3]:   
            control=control+1
            if control==len(images):
                control=0
        elif y > button2[0] and y < button2[1] and x > button2[2] and x < button2[3]:   
            control=control-1
        elif y > button3[0] and y < button3[1] and x > button3[2] and x < button3[3]:   
            rgb=rgb+1
            if rgb==4:
                rgb=0
        elif y > button4[0] and y < button4[1] and x > button4[2] and x < button4[3]:   
            normalized_on=not normalized_on

# create a window and attach a mousecallback and a trackbar
cv2.namedWindow('Control')
cv2.setMouseCallback('Control',process_click)

#https://medium.com/analytics-vidhya/hand-detection-and-finger-counting-using-opencv-python-5b594704eb08

cap=cv2.VideoCapture(-1)
if not cap.isOpened():
    print('VideoCapture -1 failed')
    cap=cv2.VideoCapture(0)

if not cap.isOpened():
    print('VideoCapture 0 failed')
    cap=cv2.VideoCapture(1)
else:
    print('VideoCapture 0 successfully done')
    
if not cap.isOpened():
    raise IOError("Cannot Open Video")

def remove_red(img):
    img[:,:,2] = np.zeros([img.shape[0], img.shape[1]])
def remove_green(img):
    img[:,:,1] = np.zeros([img.shape[0], img.shape[1]])
def remove_blue(img):
    img[:,:,0] = np.zeros([img.shape[0], img.shape[1]])

def red_to_gray(img):
    img[:,:,1] = img[:,:,2]
    img[:,:,0] = img[:,:,2]
def green_to_gray(img):
    img[:,:,2] = img[:,:,1]
    img[:,:,0] = img[:,:,1]
def blue_to_gray(img):
    img[:,:,2] = img[:,:,0]
    img[:,:,1] = img[:,:,0]
    
old_frame=[]
old_frame2=[]
old_frame3=[]
while True:
    # Read the input image
    ret,frame = cap.read()
    if len(old_frame)!=0 and len(old_frame2)!=0 and len(old_frame3)!=0:
        frame=(frame+old_frame+old_frame2+old_frame3)/4
        #cv2.accumulateWeighted(frame,old_frame,0.5)
    old_frame3=old_frame2
    old_frame2=old_frame
    old_frame=frame
    #old_frame2=old_frame.copy().astype(float)
    frame=frame.copy().astype('uint8')
    
    frame = cv2.blur(frame, (3,3))
    inverted = 255 - frame
    
    ycrcb=cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
    
    #Make ycrcb more intuitive 
    #x = ycrcb[:,:,1]
    #ycrcb[:,:,1]=ycrcb[:,:,0]
    #ycrcb[:,:,0]=x
    
    # Convert into grayscale
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    hsvim = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    skin_test=hsvim.copy().astype('uint8')
    remove_red(skin_test)
    
    #https://numpy.org/doc/stable/reference/generated/numpy.maximum.html
    skin_test[:,:,0]=np.maximum(skin_test[:,:,1],skin_test[:,:,2])
    skin_test[:,:,1]=skin_test[:,:,0]
    skin_test[:,:,2]=skin_test[:,:,0]
    skin_test = cv2.cvtColor(skin_test, cv2.COLOR_RGB2GRAY)
    th_otsu,skin_test = cv2.threshold(skin_test, 0, 255, cv2.THRESH_OTSU)
    skin_test = cv2.cvtColor(skin_test, cv2.COLOR_GRAY2RGB)
    
    equally_saturate=hsvim.copy().astype('uint8')
    equally_saturate[:,:,1] = np.max(equally_saturate[:,:,1])
    equally_saturate = cv2.cvtColor(equally_saturate, cv2.COLOR_HSV2BGR)
    
    equally_hue=hsvim.copy().astype('uint8')
    equally_hue[:,:,0] = np.max(equally_hue[:,:,0])
    equally_hue = cv2.cvtColor(equally_hue, cv2.COLOR_HSV2BGR)
    #cv2.imshow('img', hsvim)
    
    #lower = np.array([110, 110, 150], dtype = "uint8")
    #upper = np.array([255, 255, 255], dtype = "uint8")
    #skinRegionHSV_ = cv2.inRange(hsvim, lower, upper)
    
    skinRegionHSV_=hsvim.copy().astype('uint8')
    skinRegionHSV_ =skinRegionHSV_[:,:,0]
    #skinRegionHSV_ = cv2.normalize(skinRegionHSV_, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    if normalized_on:
        th_otsu,skinRegionHSV_ = cv2.threshold(skinRegionHSV_, 0, 255, cv2.THRESH_OTSU)
        #skinRegionHSV_ = 255-cv2.adaptiveThreshold(skinRegionHSV_,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,3,2)
    
    skinRegionHSV = cv2.cvtColor(skinRegionHSV_, cv2.COLOR_GRAY2RGB)
    
    masked_from_hsvim = cv2.bitwise_and(hsvim, hsvim, mask=skinRegionHSV_)
    masked_from_frame = cv2.bitwise_and(frame, frame, mask=skinRegionHSV_)
    
    blurred_ = cv2.blur(skinRegionHSV_, (2,2))
    
    blurred = cv2.cvtColor(blurred_, cv2.COLOR_GRAY2RGB)
    
    ret,thresh = cv2.threshold(blurred_,0,255,cv2.THRESH_BINARY)
    
    thresh_before_contour= cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)
    
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours)>0:
        contours = max(contours, key=lambda x: cv2.contourArea(x))
    
        thresh_before_convexHull= cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)
    
        cv2.drawContours(thresh_before_convexHull, [contours], -1, (255,255,0), 2)

        hull = cv2.convexHull(contours)
    
        before_defects= cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)

        cv2.drawContours(before_defects, [hull], -1, (0, 255, 255), 2)

        convexhull = cv2.convexHull(contours, returnPoints=False)
    
        defects = cv2.convexityDefects(contours, convexhull)
        
        if defects is not None:
            for i in range(defects.shape[0]):
                s,e,f,d = defects[i,0]
                start = tuple(contours[s][0])
                end = tuple(contours[e][0])
                far = tuple(contours[f][0])
                cv2.line(thresh,start,end,[0,255,0],2)
                cv2.circle(thresh,far,5,[0,0,255],-1)
       
    thresh= cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)
    
    displayed=locals()[images[control]]
    
    rgb_str='BGR'
    #index 0 is for blue
    #index 1 is for green
    #index 2 is for red
    if rgb==1 or rgb==4 or rgb==5:
        red_to_gray(displayed)
        rgb_str="red_to_gray"
    if rgb==2 or rgb==4 or rgb==6:
        green_to_gray(displayed)
        rgb_str="green_to_gray"
    if rgb==3 or rgb==5 or rgb==6:
        blue_to_gray(displayed)
        rgb_str="blue_to_gray"
    
    if normalized_on:
        displayed = cv2.normalize(displayed, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        displayed = cv2.Canny(image=displayed, threshold1=100, threshold2=200) # Canny Edge Detection
        
    drawButton(displayed,'next',button)
    drawButton(displayed,'prev',button2)
    drawButton(displayed,'RGB',button3)
    drawButton(displayed,'Normalize' if not normalized_on else 'stop normalization',button4)
    
    cv2.putText(displayed, 'Report: control='+images[control]+':'+rgb_str,(10, 18), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0))
                   
    cv2.imshow('Control', displayed)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

VideoCapture 0 successfully done
